## Main Goals: 
Create a cleaned development dataset that can be used to complete the modeling step of this project
- Create dummy or indicator features for categorical variables  
- Standardize the magnitude of numeric features using a scaler  
- Split into testing and training datasets

## 1. Imports packages and load the data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer

In [2]:
jobs = pd.read_csv('data/jobs_data_step3_features.csv')

In [3]:
print(jobs.info())
print('There are {} observations and {} features in this dataset.'.format(jobs.shape[0],jobs.shape[1]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2088 entries, 0 to 2087
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rating             1980 non-null   float64
 1   Company Name       2088 non-null   object 
 2   Location           2088 non-null   object 
 3   Size               2047 non-null   object 
 4   Type of ownership  2073 non-null   object 
 5   Industry           1899 non-null   object 
 6   Sector             1899 non-null   object 
 7   Revenue            1474 non-null   object 
 8   Years Founded      1592 non-null   float64
 9   Min_Salary         2088 non-null   int64  
 10  Max_Salary         2088 non-null   int64  
 11  Seniority          2088 non-null   object 
 12  HQ_Same            2088 non-null   int64  
 13  SQL                2088 non-null   int64  
 14  Excel              2088 non-null   int64  
 15  Python             2088 non-null   int64  
 16  Tableau            2088 

In [4]:
jobs.head()

,Rating,Company Name,Location,Size,Type of ownership,Industry,Sector,Revenue,Years Founded,Min_Salary,Max_Salary,Seniority,HQ_Same,SQL,Excel,Python,Tableau,R,SAS
0,3.2,Vera Institute of Justice,NY,201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),59.0,37,66,Not specified,1,1,0,1,0,1,0
1,3.8,Visiting Nurse Service of New York,NY,10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),127.0,37,66,Not specified,1,1,1,0,0,1,1
2,3.4,Squarespace,NY,1001 to 5000 employees,Company - Private,Internet,Information Technology,NaN,17.0,37,66,Senior,1,1,1,0,1,0,0
3,4.1,Celerity,NY,201 to 500 employees,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),18.0,37,66,Not specified,0,1,0,0,1,0,0
4,3.9,FanDuel,NY,501 to 1000 employees,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),11.0,37,66,Not specified,1,1,1,1,0,0,0


## 2.Train/Test Split

In [5]:
# Split data to predict Min_Salary
X_train, X_test, y_train1, y_test1 = train_test_split(jobs.drop(columns=['Min_Salary','Max_Salary']), 
                                                    jobs.Min_Salary, test_size=0.3, random_state=47)
# Split data to predict Max_Salary
X_train, X_test, y_train2, y_test2 = train_test_split(jobs.drop(columns=['Min_Salary','Max_Salary']), 
                                                    jobs.Max_Salary, test_size=0.3, random_state=47)

In [6]:
X_train.shape, X_test.shape

((1461, 17), (627, 17))

## 3.Initial models - Pipelines 

Create pipelines to： 
- try using different ways to impute missing values
- scale the data to zero mean and unit variance
- train a linear regression model  
Note: The EDA that we did earlier suggests a positive linear correlation between the features 'Size' and 'Years Founded'. Therefore, we will try using 'Size' to impute the missing values in the column of 'Years Founded' here.

In [7]:
# check the number of missing values for each column
jobs.isnull().sum().sort_values(ascending=False)

Revenue              614
Years Founded        496
Industry             189
Sector               189
Rating               108
Size                  41
Type of ownership     15
Company Name           0
Location               0
SAS                    0
R                      0
Max_Salary             0
Seniority              0
HQ_Same                0
SQL                    0
Excel                  0
Python                 0
Tableau                0
Min_Salary             0
dtype: int64

In [8]:
# group jobs data by size and compare Years Founded
years_bySize = jobs.groupby('Size')['Years Founded'].mean()
years_bySize.sort_values(ascending=True)

Size
1 to 50 employees          15.357143
51 to 200 employees        16.078689
201 to 500 employees       24.837321
501 to 1000 employees      26.572917
1001 to 5000 employees     36.654088
5001 to 10000 employees    50.321839
10000+ employees           90.179837
Name: Years Founded, dtype: float64

It seems obvious that the larger the company, the longest history it has.

In [9]:
# Impute Years_Founded using Size
# If Size is also missing, impute Years Founded by mean of Years Founded

#jobs.loc[jobs.Size.isnull()==True,'Years Founded'].fillna(X_train['Years Founded'].mean(),inplace=True)
jobs.loc[jobs.Size.isnull()==True,'Years Founded']=X_train['Years Founded'].mean()
jobs['Years Founded']=jobs.dropna(subset=['Size'],axis=0).groupby("Size")['Years Founded'].transform(lambda x: x.fillna(x.mean()))
jobs['Years Founded'].value_counts()

15.357143     235
16.078689     115
12.000000      80
18.000000      54
20.000000      54
             ... 
98.000000       1
163.000000      1
167.000000      1
94.000000       1
125.000000      1
Name: Years Founded, Length: 166, dtype: int64

In [10]:
# drop the feature 'Size'
jobs.drop('Size',axis=1,inplace=True)

#### Impute numeric missing values with median & Impute categorical data with most common values


In [16]:
jobs.columns

Index(['Rating', 'Company Name', 'Location', 'Type of ownership', 'Industry',
       'Sector', 'Revenue', 'Years Founded', 'Min_Salary', 'Max_Salary',
       'Seniority', 'HQ_Same', 'SQL', 'Excel', 'Python', 'Tableau', 'R',
       'SAS'],
      dtype='object')

In [20]:
features = jobs[['Rating', 'Years Founded', 'HQ_Same', 'SQL', 'Excel','Python','Tableau','R','SAS',
                 'Company Name','Location','Type of ownership','Industry','Sector','Revenue','Seniority']].copy()
categorical_features = features.dtypes == 'object'
numerical_features = ~categorical_features #features.dtypes == 'float'
preprocess = make_column_transformer(
    (numerical_features, make_pipeline(SimpleImputer(strategy='median'), StandardScaler())),
    (categorical_features, make_pipeline(SimpleImputer(strategy="most_frequent"),OneHotEncoder()))
)

In [21]:
pipe = make_pipeline(
    preprocess,
    LinearRegression()
)

In [22]:
pipe.fit(X_train, y_train1)

C:\Users\Abby\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [23]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

AttributeError: 'ColumnTransformer' object has no attribute '_n_features'

In [24]:
X_train.median()

Rating            3.7
Years Founded    23.5
HQ_Same           0.0
SQL               1.0
Excel             0.0
Python            0.0
Tableau           0.0
R                 0.0
SAS               0.0
dtype: float64

In [25]:
X_tr = X_train.fillna(X_train.median())
X_te = X_test.fillna(X_train.median())

In [26]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

ValueError: could not convert string to float: 'Heartland Payment Systems'